In [1]:
df=spark.sql("select * from user_erin.rating")
(train, test) = df.randomSplit([0.8, 0.2],seed = 11)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
9,None,pyspark,idle,Link,,erin,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 机器学习：ALS模型的调参与预测
- ALS算法是基于模型的推荐算法。其基本思想是对稀疏矩阵进行模型分解，评估出缺失项的值，以此来得到一个基本的训练模型。然后依照此模型可以针对新的用户和物品数据进行评估。ALS采用交替的最小二乘法来算出缺失项。
- 运用TrainValidationSplit（TVS）来进行参数调优。TVS对于每个参数组合只进行一次评估，当训练数据集足够大时，开销较小，并且产生的结果可靠。TVS会创建单个 (training, test) pair。它使用trainRatio参数将数据集split成两部分。例如： trainRatio=0.8，TVS会生成一个训练集（80%）和一个测试集（20%）。TVS最后会使用Estimator、以及最好的ParamMap，对整个数据集进行拟合。
- 运用param_grid方程来定义TVS检测的参数，在以下单元格，regParam有四个参数选择，所以最后一共有四种参数组合模型被测试。
- 如果您的数据集较小，并想测试多种参数组合，可以在参数列表里加入希望测试的数值。
- maxIter 设置的越大结果越精确，但是设置的越大也就意味着越耗时；一般建议10-20。
- regParam:惩罚函数的因数，是ALS的正则化参数。如果设置很大就可以防止过拟合问题，如果设置很小，就不会有防止过拟合的功能了。
- rank设置范围为10-100


In [2]:
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
als= ALS(userCol="user",itemCol="movie",ratingCol="rating",nonnegative=True,implicitPrefs=False, coldStartStrategy="drop",maxIter=10)
als_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="rating",metricName="rmse")
param_grid = ParamGridBuilder() \
            .addGrid(als.regParam, [ .01]) \
            .build()
#, .05, .1, .15
tvs = TrainValidationSplit(estimator=als,
                           estimatorParamMaps=param_grid,
                           evaluator=als_evaluator,
                           trainRatio = 0.8)
model=tvs.fit(train)
model=model.bestModel
als_prediction=model.transform(test)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
evaluator = RegressionEvaluator(metricName="mae", labelCol="rating",
                                predictionCol="prediction")

mae = evaluator.evaluate(als_prediction)
print("mean absolute error = " + str(mae))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

mean absolute error = 1.04900298892291

## 机器学习：预测储存与模型导出
- 您可以通过hdfs路径以及存入模型的模型种类，将模型导出进行重复使用

In [4]:
als_prediction.write.saveAsTable("user_erin.rating_al", format="orc", mode="overwrite")
model.write().overwrite().save("hdfs:///user/erin/data/mllib/alsmodel") 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
spark.stop()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…